In [1]:
%%configure -f
{
    "jars":["/home/isam/ApproxIoT/jars/magellan-1.0.5-s_2.11.jar"],
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0,com.esri.geometry:esri-geometry-api:1.2.1,commons-io:commons-io:2.6",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11",
        "spark.dynamicAllocation.enabled": false
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
18,None,spark,idle,,,


In [2]:
/*
 * Author: Isam Al Jawarneh
 * Updated: October 25, 2022
 * Purpose: PREPROCESSING: the following notebook generates geohashes for millions of georeferenced data points 
 *         , in addition to corresponding neighborhoods and stores the tuples in a CSV file for further processing 
 */

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
19,None,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
//importing required libraries --- Magellan is a local jar file

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import util.control.Breaks._
import org.apache.spark.sql.streaming.StreamingQueryListener
//import org.apache.spark.util.random.XORShiftRandom
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.SQLImplicits
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.ForeachWriter
import magellan._
import magellan.index.ZOrderCurve
import magellan.{Point, Polygon}

import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{
  DoubleType,
  StringType,
  StructField,
  StructType
}
import org.apache.spark.sql.streaming._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.execution.streaming.MemoryStream
import org.apache.spark.sql.functions.{collect_list, collect_set}
import org.apache.spark.sql.SQLContext
import org.apache.log4j.{Level, Logger}
import scala.collection.mutable
import scala.concurrent.duration.Duration
import java.io.{BufferedWriter, FileWriter}
import org.apache.commons.io.FileUtils
import java.io.File
import scala.collection.mutable.ListBuffer
import java.time.Instant
//import org.apache.spark.util.CollectionAccumulator
import org.apache.spark.sql.DataFrame
import org.apache.kafka.clients.consumer.ConsumerRecord
import org.apache.kafka.common.serialization.StringDeserializer

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import util.control.Breaks._
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.SQLImplicits
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.ForeachWriter
import magellan._
import magellan.index.ZOrderCurve
import magellan.{Point, Polygon}
import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{DoubleType, StringType, StructField, StructType}
import org.apache.spark.s

In [5]:
//defining the schema of the data that we are reading from a CSV file
//field names and arrangement are equivalent to those in the CSV file heading
val schemaShenzhenShort = StructType(Array(
    StructField("id", StringType, false),
    StructField("lat", DoubleType, false),
    StructField("lon", DoubleType, false),
    StructField("time", StringType, false),
    StructField("speed", DoubleType, false)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schemaShenzhenShort: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,false), StructField(lat,DoubleType,false), StructField(lon,DoubleType,false), StructField(time,StringType,false), StructField(speed,DoubleType,false))


In [6]:
//parameters: geohash precision. Other common values: 25, 35

val precision = 30


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

precision: Int = 30


In [7]:
// user defined function for converting the ZOrderCurve into the corresponding geohash
val geohashUDF = udf{(curve: Seq[ZOrderCurve]) => curve.map(_.toBase32())}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

geohashUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(StringType,true),Some(List(ArrayType(org.apache.spark.sql.types.ZOrderCurveUDT@139992bd,true))))


In [8]:
//reading the CSV tuples and adding a geohash column containing geohash value for each tuple
val transformationStream1 = spark.read.option("inferSchema", "true").option("header", "true").csv("/home/isam/Desktop/spatial/data/china/points/guang.csv")

val ridesGeohashed = transformationStream1.withColumn("point", point($"lat",$"lon")).withColumn("index", $"point" index  precision).withColumn("geohashArray", geohashUDF($"index.curve")).select($"id",$"lat",$"lon",$"time", $"point",$"geohashArray",$"speed")
val dataStream1 = ridesGeohashed.explode("geohashArray", "geohash") { a: mutable.WrappedArray[String] => a }
val transformationStream = dataStream1.select("id","lat","lon","time","point","speed", "geohash")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

transformationStream1: org.apache.spark.sql.DataFrame = [id: int, lat: double ... 3 more fields]
ridesGeohashed: org.apache.spark.sql.DataFrame = [id: int, lat: double ... 5 more fields]
dataStream1: org.apache.spark.sql.DataFrame = [id: int, lat: double ... 6 more fields]
transformationStream: org.apache.spark.sql.DataFrame = [id: int, lat: double ... 5 more fields]


In [9]:
transformationStream.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- point: point (nullable = false)
 |-- speed: integer (nullable = true)
 |-- geohash: string (nullable = true)



In [10]:
transformationStream.createOrReplaceTempView("queryTable_static")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
transformationStream.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+------------------+-------------------+--------------------+-----+-------+
| id|               lat|               lon|               time|               point|speed|geohash|
+---+------------------+------------------+-------------------+--------------------+-----+-------+
|  0|114.03179899999999|22.524798999999998|2014-10-22 04:54:30|Point(114.0317989...|   42| ws104u|
|  0|        114.038696|           22.5315|2014-10-22 04:54:37|Point(114.038696,...|   52| ws105j|
+---+------------------+------------------+-------------------+--------------------+-----+-------+
only showing top 2 rows



In [12]:
//static: counting the number of tuples
val count = spark.sql("SELECT count(speed) from queryTable_static")
count.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

count: org.apache.spark.sql.DataFrame = [count(speed): bigint]
+------------+
|count(speed)|
+------------+
|     1155653|
+------------+



In [13]:
//counting distinct geohashes in the CSV file (after adding the geohash column)
val df2 = transformationStream.select(countDistinct("geohash"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df2: org.apache.spark.sql.DataFrame = [count(DISTINCT geohash): bigint]


In [14]:
//counting distinct geohashes in the CSV file (after adding the geohash column)

df2.show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|count(DISTINCT geohash)|
+-----------------------+
|1561                   |
+-----------------------+



In [15]:
//  a function that takes the geohash precision and the polygons GeoJSON file
//it generates the geohash covering for all the polygons in the GeoJSON file and explodes them
// so that for each polygon, there are many records with each record containing one geohash value of the cover

def geohashedNeighborhoods(geohashPrecision: Int, filePath: String): DataFrame = 

{

import spark.implicits._
/*preparing the neighborhoods table (static table) .... getting geohashes covering for every neighborhood and 
exploding it, so that each neighborhood has many geohashes*/

// this will be executed only one time - batch mode 
val rawNeighborhoods = spark.sqlContext.read.format("magellan").option("type", "geojson").load(filePath).select($"polygon", $"metadata"("NAME").as("neighborhood")).cache()

val neighborhoods = rawNeighborhoods.withColumn("index", $"polygon" index geohashPrecision).select($"polygon", $"index", 
      $"neighborhood").cache()
    print(neighborhoods.count())

val zorderIndexedNeighborhoods = neighborhoods.withColumn("index", explode($"index")).select("polygon", "index.curve", "index.relation","neighborhood")
val geohashedNeighborhoods= neighborhoods.withColumn("geohashArray", geohashUDF($"index.curve"))

val explodedgeohashedNeighborhoods = geohashedNeighborhoods.explode("geohashArray", "geohash") { a: mutable.WrappedArray[String] => a }

//unit testing: explodedgeohashedNeighborhoods.show(10)


explodedgeohashedNeighborhoods

}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

geohashedNeighborhoods: (geohashPrecision: Int, filePath: String)org.apache.spark.sql.DataFrame


In [38]:
//reading the neighborhoods file on the form of GeoJSON
val geohashedNeigboors = geohashedNeighborhoods(precision,"/home/isam/Desktop/spatial/data/china/neigh_Douglas_shapefiles/converted2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7geohashedNeigboors: org.apache.spark.sql.DataFrame = [polygon: polygon, index: array<struct<curve:zordercurve,relation:string>> ... 3 more fields]


In [39]:
//dropping duplicate values
geohashedNeigboors.dropDuplicates("geohash").show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+------------+--------------------+-------+
|             polygon|               index|neighborhood|        geohashArray|geohash|
+--------------------+--------------------+------------+--------------------+-------+
|magellan.Polygon@...|[[ZOrderCurve(113...|         宝安区|[ws0chk, ws0chm, ...| ws0buz|
|magellan.Polygon@...|[[ZOrderCurve(113...|         宝安区|[ws0chk, ws0chm, ...| ws0bzp|
+--------------------+--------------------+------------+--------------------+-------+
only showing top 2 rows



In [40]:
//change the name to remove the ambiguity that may appear as both points and polygons have geohash field name
//so, we name the geohash field in the neighborhoods file as geohashn (for geohash neighborhood)
val newDF = geohashedNeigboors.withColumnRenamed("geohash","geohashn")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

newDF: org.apache.spark.sql.DataFrame = [polygon: polygon, index: array<struct<curve:zordercurve,relation:string>> ... 3 more fields]


In [41]:
//applying the filter-and-refinement approach from Spark's Magellan to join points and polygons
// returning to which polygon (neighborhood, district, etc.,) each point belongs
// MBR-join then point-in-polygon test for candidates
val samplepointDF_SSS_join = transformationStream.join(newDF,newDF("geohashn")===transformationStream("geohash")).where($"point" within $"polygon")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

samplepointDF_SSS_join: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, lat: double ... 10 more fields]


In [42]:
//counting distinct geohash values in the neighborhoods file
//in other terms, how much are the geohashes covering the study area (all polygons)
val df3 = samplepointDF_SSS_join.select(countDistinct(newDF("geohashn")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df3: org.apache.spark.sql.DataFrame = [count(DISTINCT geohashn): bigint]


In [43]:
df3.show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|count(DISTINCT geohashn)|
+------------------------+
|1420                    |
+------------------------+



In [44]:
//how many points do we have after the join in the new dataframe, consisting of the geohash value and the 
//neighborhood of each tuple from the CSV file
samplepointDF_SSS_join.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res53: Long = 1153389


In [45]:
// the new schema shows that the new dataframe contains the geohash value and the neighborhood for each tuple 

samplepointDF_SSS_join.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- point: point (nullable = false)
 |-- speed: integer (nullable = true)
 |-- geohash: string (nullable = true)
 |-- polygon: polygon (nullable = true)
 |-- index: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- curve: zordercurve (nullable = false)
 |    |    |-- relation: string (nullable = false)
 |-- neighborhood: string (nullable = true)
 |-- geohashArray: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- geohashn: string (nullable = true)



// selecting the same fields (corresponding to the original file CSV schema ) and adding the geohash field
val printDF = samplepointDF_SSS_join.select($"id",$"lat",$"lon",$"time",$"speed")

In [ ]:
// selecting part of the dataframe (containing the new geohash and neighborhood) to store in a CSV file
val printDF = samplepointDF_SSS_join.select($"id",$"lat",$"lon",$"time",$"speed")

printDF.count()

printDF.show(2)

In [49]:
//samplepointDF_SSS_join.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
//storing the resulting dataframe in a new CSV file
// to serve it to another stage in the analytics pipeline
printDF.coalesce(1).write.option("header",true).csv("/home/isam/Desktop/spatial/data/china/datacsv/data2.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…